In [1]:
# Import Dependencies
import requests
import json
import pandas as pd
import time
import numpy as np
from config import yelp_key

In [2]:
# API Base URL
base_url = "https://api.yelp.com/v3/businesses/search?"

In [3]:
# API Headers
headers = {
    "accept" : "application/json",
    "Authorization" : "Bearer " + yelp_key 
}

In [4]:
# Create List of Categories (manually located)
categories = ['beaches','hanggliding','horsebackriding',
             'hiking','hot_air_balloons',
              'paddleboarding','parasailing',
              'sailing','snorkeling','ziplining', 
              'spas','hotsprings','massage','tours'
             ]

len(categories)

21

In [21]:
# Import Lat/Long Database
csv_path = "../Resources/CitiesWGeolocation.csv"
locations_df = pd.read_csv(csv_path)
locations_df.head()

,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,0,Lexington,Kentucky,United States,38.046407,-84.497039
1,1,San Diego,California,United States,32.717420,-117.162773
2,2,Cook Islands,NaN,Cook Islands,-19.996972,-157.785871
3,3,Park City,Utah,United States,40.646092,-111.497996
4,4,Newcastle Upon Tyne,England,United Kingdom,54.973847,-1.613157


In [22]:
# Normalize capitalization
locations_df = locations_df.rename(columns = {"Latitude": "latitude","Longitude":"longitude","Unnamed: 0":"locationID"})

In [7]:
# Gather Lists of Latitudes and Longitudes
latitudes_list = []
longitudes_list = []
location_list = []
for i in range(locations_df.shape[0]):
    latitudes_list.append(locations_df.loc[i].at['latitude'])
    longitudes_list.append(locations_df.loc[i].at['longitude'])
    location_list.append(locations_df.loc[i].at['locationID'])
print(latitudes_list)
print(longitudes_list)

[38.0464066, 32.7174202, -19.99697155, 40.646092100000004, 54.97384739999999, -27.468968199999996, -28.0402165, 34.244058700000004, -36.718804999999996, 49.5041747, 42.64873625, 37.2395367, -32.728465, 17.223472100000002, 25.265347100000003, 50.67108245, 21.721746, 36.508976000000004, 32.079007399999995, -16.484598300000002, -33.928992, -19.1421421, -33.934444, -28.648333299999997, -45.0321923, 33.77217945, -6.166490799999999, -20.2759451, 45.437190799999996, 19.70318225, -4.6574977, -28.002373100000003, 44.4643768, 31.6258257, 37.9374939, 49.8879177, 43.10656029999999, 30.2711224, 15.2214956, 43.4832523, -33.953177600000004, 26.9154576, 13.1500331, 20.169626800000003, 51.53882410000001, 60.39430550000001, 24.578720999999998, -34.61341495, 34.862942600000004, 18.1850507, 12.51756625, -35.14181285, -34.4175, -33.8611665, -16.9206657, 44.958452799999996, -26.6544338, -34.427808299999995, 51.08668970000001, 44.279621, -37.8142454, -31.955893300000003, 51.453802200000005, 44.841225, 27.175

In [8]:
# Gather List of URLs
url_list = []
lat_lon_list = []
cat_list = []
location_ID_list = []
for i in range(len(latitudes_list)):
        for x in range(len(categories)):
            latitude = latitudes_list[i]
            longitude = longitudes_list[i]
            category = categories[x]
            location = location_list[i]
            # Limit radius to appx 15 miles from the coordinates
            complete_url = base_url + "latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&radius=40000&categories=" + category + "&sort_by=rating"
            url_list.append(complete_url)
            lat_lon_list.append([latitude,longitude])
            cat_list.append(category)
            location_ID_list.append(location)
# Confirm list of API calls is less than 5,000 daily limit
print(len(url_list))
url_list

4662


['https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=beaches&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hanggliding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=horsebackriding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hiking&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hot_air_balloons&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=paddleboarding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=parasailing&sort_by=rating',
 'https://api.yelp.com/v

In [9]:
# Create API Calls (separate the blank list to a new cell so we can do numerous calls if needed)
data_list=[]

In [11]:
record_count = 1
set_count = 1
url_errors = []


for i in range(len(url_list)):
    if (i%100 == 0 and i >=100):
        set_count += 1
        record_count = 0
        
    print("processing Record %s of Set %s" %(record_count, set_count))    
    
    record_count += 1

    response = requests.get(url_list[i],headers=headers)
    data = response.json()
    try:
        # Calculate Average Rating and Review Count
        rating_sum = 0
        review_sum = 0
        # setting the range with an initial number so it can be changed if an error pops up midway through
        for x in range(len(data["businesses"])):
            rating_sum += data["businesses"][x]["rating"]
            review_sum += data["businesses"][x]["review_count"]
        rating_avg = rating_sum / len(data["businesses"])
        review_avg = review_sum / len(data["businesses"])
    
        # Add the desired data to a dictionary
        data_dict = {
            "locationID" : location_ID_list[i],
            "latitude" : lat_lon_list[i][0] ,
            "longitude": lat_lon_list[i][1],
            "category" : cat_list[i],
            "results_total" : len(data["businesses"]),
            "avg_rating" : rating_avg,
            "avg_review_count" : review_avg
        }
        # Append the dictionary to a list
        data_list.append(data_dict)
        time.sleep(1)
    except:
        print("********URL error: Skipping...********")
        print(data)
        try:
            url_errors.append({"locationID":location_ID_list[i],"url":url_list[i],"latitude":data["region"]["center"]["latitude"],
                           "longitude":data["region"]["center"]["longitude"]})
        except:
            url_errors.append({"locationID":location_ID_list[i],"url":url_list[i],"latitude":"NAN",
                           "longitude":"NAN"})
        pass

processing Record 1 of Set 1
processing Record 2 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 3 of Set 1
processing Record 4 of Set 1
processing Record 5 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 6 of Set 1
processing Record 7 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 8 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 9 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 1

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -1.6131572, 'latitude': 54.97384739999999}}}
processing Record 92 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -1.6131572, 'latitude': 54.97384739999999}}}
processing Record 93 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -1.6131572, 'latitude': 54.97384739999999}}}
processing Record 94 of Set 1
processing Record 95 of Set 1
processing Record 96 of Set 1
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -1.6131572, 'latitude': 54.97384739999999}}}
processing Record 97 of Set 1
processing Record 98 of Set 1
processing Record 99 of Set 1
processing Record 100 of Set 1
processing Record 0 of Set 2
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitu

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 142.22095086226386, 'latitude': -36.718804999999996}}}
processing Record 82 of Set 2
processing Record 83 of Set 2
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 142.22095086226386, 'latitude': -36.718804999999996}}}
processing Record 84 of Set 2
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 142.22095086226386, 'latitude': -36.718804999999996}}}
processing Record 85 of Set 2
processing Record 86 of Set 2
processing Record 87 of Set 2
processing Record 88 of Set 2
processing Record 89 of Set 2
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -115.062867, 'latitude': 49.5041747}}}
processing Record 90 of Set 2
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -

processing Record 50 of Set 3
processing Record 51 of Set 3
processing Record 52 of Set 3
processing Record 53 of Set 3
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 151.9380100885447, 'latitude': -32.728465}}}
processing Record 54 of Set 3
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 151.9380100885447, 'latitude': -32.728465}}}
processing Record 55 of Set 3
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 151.9380100885447, 'latitude': -32.728465}}}
processing Record 56 of Set 3
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 151.9380100885447, 'latitude': -32.728465}}}
processing Record 57 of Set 3
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 151.9380100885447, 'latitude': -32.728465}}}
processing

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 55.292491399999996, 'latitude': 25.265347100000003}}}
processing Record 10 of Set 4
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 55.292491399999996, 'latitude': 25.265347100000003}}}
processing Record 11 of Set 4
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 55.292491399999996, 'latitude': 25.265347100000003}}}
processing Record 12 of Set 4
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 55.292491399999996, 'latitude': 25.265347100000003}}}
processing Record 13 of Set 4
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 55.292491399999996, 'latitude': 25.265347100000003}}}
processing Record 14 of Set 4
********URL error: Skipping...********
{'businesses': [

...

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -75.6901106, 'latitude': 45.4208777}}}
processing Record 52 of Set 44
processing Record 53 of Set 44
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -75.6901106, 'latitude': 45.4208777}}}
processing Record 54 of Set 44
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -75.6901106, 'latitude': 45.4208777}}}
processing Record 55 of Set 44
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -75.6901106, 'latitude': 45.4208777}}}
processing Record 56 of Set 44
processing Record 57 of Set 44
processing Record 58 of Set 44
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -75.6901106, 'latitude': 45.4208777}}}
processing Record 59 of Set 44
processing Record 60 of Set 44
p

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -17.447938, 'latitude': 14.693425}}}
processing Record 21 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -17.447938, 'latitude': 14.693425}}}
processing Record 22 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -17.447938, 'latitude': 14.693425}}}
processing Record 23 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -17.447938, 'latitude': 14.693425}}}
processing Record 24 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -17.447938, 'latitude': 14.693425}}}
processing Record 25 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -17.447938, 'latitude': 14.693

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.250168700000001, 'latitude': 55.861155000000004}}}
processing Record 81 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.250168700000001, 'latitude': 55.861155000000004}}}
processing Record 82 of Set 45
processing Record 83 of Set 45
processing Record 84 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.250168700000001, 'latitude': 55.861155000000004}}}
processing Record 85 of Set 45
processing Record 86 of Set 45
processing Record 87 of Set 45
processing Record 88 of Set 45
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.250168700000001, 'latitude': 55.861155000000004}}}
processing Record 89 of Set 45
processing Record 90 of Set 45
processing Record 91 of Set 45
processing Record 92 of

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 61 of Set 46
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 62 of Set 46
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 63 of Set 46
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 64 of Set 46
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 65 of Set 46
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center':

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 33 of Set 47
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 34 of Set 47
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 35 of Set 47
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 36 of Set 47
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 37 of Set 47
********URL error: Skipping...********
{'businesse

In [12]:
# convert the list of dictionaries to a dataframe
yelp_df = pd.DataFrame(data_list)   
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,38.046407,-84.497039,beaches,2,4.75,3.500
1,0,38.046407,-84.497039,horsebackriding,16,4.50,13.375
2,0,38.046407,-84.497039,hiking,8,4.50,8.500
3,0,38.046407,-84.497039,paddleboarding,2,4.75,16.000
4,0,38.046407,-84.497039,ziplining,20,5.00,8.450


In [13]:
# Convert the list of errors into a dataframe to view any similarities
errors_df = pd.DataFrame(url_errors)
errors_df.head()

,locationID,url,latitude,longitude
0,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
1,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
2,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
3,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
4,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039


In [14]:
# determine if there are any missing reviews. All 21 categories are accounted for
yelp_df["category"].value_counts()

transportation      163
ziplining           163
restaurants         160
nightlife           149
hotels              147
tours               143
massage             139
spas                138
bedbreakfast        133
beaches             122
hiking              122
horsebackriding     114
resorts             113
sailing              76
paddleboarding       57
skiresorts           47
hot_air_balloons     47
snorkeling           29
hanggliding          18
hotsprings           13
parasailing          11
Name: category, dtype: int64

In [15]:
# Check how many categories successfully returned for each of the locations that returned.
# We are missing many locations, as the length should be 237. None of our locations have the full 21 activities either.
yelp_df["latitude"].nunique()

164

In [23]:
# Separate out all of the values that have less than 5 actual activities returned successfully
activity_df = pd.DataFrame(yelp_df.loc[(yelp_df["category"] != "transportation") & (yelp_df["category"] !="restaurants") &
                                       (yelp_df["category"] != "hotels")         & (yelp_df["category"] != "nightlife")])
value_counts = pd.DataFrame(activity_df[["locationID","latitude","longitude"]].groupby(["latitude","longitude"]).size().sort_values(ascending=False))
value_counts = value_counts.rename(columns={0:"activityCount","longitude":"test"})
value_counts = value_counts.loc[(value_counts["activityCount"] < 5)]
value_counts = value_counts.reset_index(inplace=False)
value_counts.head()

,latitude,longitude,activityCount
0,12.517566,-69.981864,4
1,55.399722,10.385210,3
2,-33.861167,121.888516,3
3,-8.456018,115.270386,3
4,13.825049,-60.975036,2


In [18]:
# Filter the list of results to only include locations with five or more actual activities
longitudes_list = value_counts["longitude"].tolist()
check_five = yelp_df[~yelp_df["longitude"].isin(longitudes_list)]
check_five

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,38.046407,-84.497039,beaches,2,4.7500,3.5000
1,0,38.046407,-84.497039,horsebackriding,16,4.5000,13.3750
2,0,38.046407,-84.497039,hiking,8,4.5000,8.5000
3,0,38.046407,-84.497039,paddleboarding,2,4.7500,16.0000
4,0,38.046407,-84.497039,ziplining,20,5.0000,8.4500
...,...,...,...,...,...,...,...
2096,233,46.201756,6.146601,skiresorts,15,4.2000,2.0000
2097,233,46.201756,6.146601,tours,16,3.8125,2.8125
2098,233,46.201756,6.146601,transportation,20,5.0000,7.8500
2099,233,46.201756,6.146601,restaurants,20,5.0000,6.8000


In [25]:
# Create a dataframe of all null values in the yelp dataframe
null_df = pd.merge(check_five,locations_df,how="right",on=["latitude","longitude","locationID"])
null_df = null_df[null_df["category"].isna()]
null_df

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count,City,State/Province,Country
218,2,-19.996972,-157.785871,NaN,NaN,NaN,NaN,Cook Islands,NaN,Cook Islands
219,8,-36.718805,142.220951,NaN,NaN,NaN,NaN,Grampians,Victoria,Australia
220,10,42.648736,18.094680,NaN,NaN,NaN,NaN,Dubrovnik,Croatia,Croatia
221,13,17.223472,-61.955461,NaN,NaN,NaN,NaN,Antigua and Barbuda,NaN,Antigua and Barbuda
222,14,25.265347,55.292491,NaN,NaN,NaN,NaN,Dubai,United Arab Emirates,United Arab Emirates
...,...,...,...,...,...,...,...,...,...,...
425,223,33.895920,35.478430,NaN,NaN,NaN,NaN,Beirut,NaN,Lebanon
426,224,47.374449,8.541042,NaN,NaN,NaN,NaN,Zurich,NaN,Switzerland
427,225,46.201756,6.146601,NaN,NaN,NaN,NaN,Geneva,NaN,Switzerland
428,226,35.898982,14.513676,NaN,NaN,NaN,NaN,Valletta,NaN,Malta


In [26]:
# Export list of all locations with 5+ activities
yelp_df.to_csv("../Resources/yelpdata.csv")

In [27]:
# Export list of all locations with 0-4 activities
null_df.to_csv("../Resources/missingActivities.csv")

In [28]:
check_five['category'].unique()

array(['beaches', 'horsebackriding', 'hiking', 'paddleboarding',
       'ziplining', 'spas', 'massage', 'bedbreakfast', 'hotels',
       'resorts', 'tours', 'transportation', 'restaurants', 'nightlife',
       'hanggliding', 'hot_air_balloons', 'parasailing', 'sailing',
       'snorkeling', 'hotsprings', 'skiresorts'], dtype=object)